In [1]:
# authorize access to the drive
# This allows me to access files (Data) in my google drive
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
# This downloads Hugging Face Transformers, tools we need to fine-tune GPT-2
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-pbj2go5y
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-pbj2go5y
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 20.4MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
     |████████████████████████████████| 2.9MB 49.9MB/s 
  Created wheel for transformers: filename=transformers-3.5.0-cp36-none-any.whl size=1311439 sha256=0cf8ecb5216634e66b412f9dbfccb8b34532a8097fc0030aacee7bec6f79763b
  Stored in directory: /tmp/pip-ephem-wheel-cache-yy8sa5jq/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7f5d1df2143c0c04645c91644bd588098b226877371fee239627f150a084c42d
  Stored in directory: /root/.cach

In [3]:
# Allows me to go to the correct directory in my google drie
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [4]:
# the fine-tuning code by Hugging Face that allows us to fine-tune
# this file is already archieved after the most recent hugging face modelling se we need to replace the url
# !wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/contrib/legacy/run_language_modeling.py

--2020-11-14 12:32:37--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-11-14 12:32:37 ERROR 404: Not Found.



In [5]:
% ls

'Brian'\''s Peer Feedback Template (10.28.21).gdoc'
'Capstone Coversheet.gdoc'
'Capstone Proposal (Junior Year).gdoc'
 class/
'Dev Journal.gdoc'
 Models/
 None/
 output_description/
'Paper Drafts'/
'Punch Card.gsheet'
 run_language_modeling.py
 run_language_modeling.py.1
 run_language_modeling.py.2
 run_language_modeling.py.3
 runs/


In [6]:
# get dataset
datapath = "Models/Data"
train_path = datapath + "/train.txt"
valid_path = datapath + "/valid.txt"
output = "./output"

In [ ]:
# %%bash 
# export DIRECTORY=Models/Data/Description_Data
# ls $DIRECTORY

cached_lm_GPT2Tokenizer_1024_description_train.txt
cached_lm_GPT2Tokenizer_1024_description_train.txt.lock
cached_lm_GPT2Tokenizer_1024_description_valid.txt
cached_lm_GPT2Tokenizer_1024_description_valid.txt.lock
description_test.txt
description_train.txt
description_valid.txt


In [7]:
# Main fine-tuning code using run_language_modeling.py
%%bash
export TRAIN_FILE=Models/Data/train.txt
export TEST_FILE=Models/Data/valid.txt
export MODEL_NAME=gpt2
export OUTPUT_DIR=Models/Claims_model

# python3 run_language_modeling.py \
#     --output_dir=$OUTPUT_DIR \
#     --model_type=$MODEL_NAME \
#     --model_name_or_path=$MODEL_NAME \
#     --do_train \
#     --train_data_file=$TRAIN_FILE \
#     --do_eval \
#     --eval_data_file=$TEST_FILE \
#     --per_gpu_train_batch_size=1 \
#     --save_steps=-1 \
#     --num_train_epochs=2

python run_language_modeling.py \
    --output_dir=$OUTPUT_DIR \
    --model_type=gpt2 \
    --model_name_or_path=$MODEL_NAME \
    --do_train \
    --train_data_file=$TRAIN_FILE \
    --do_eval \
    --eval_data_file=$TEST_FILE \
    --per_gpu_train_batch_size=1 \
    --save_steps=-1 \
    --num_train_epochs=2

{'loss': 1.849806396484375, 'learning_rate': 2.7436823104693144e-05, 'epoch': 0.9025270758122743}
{'loss': 1.556890380859375, 'learning_rate': 4.873646209386281e-06, 'epoch': 1.8050541516245486}
{'epoch': 2.0}


2020-11-14 12:34:28.615376: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/14/2020 12:34:30 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2020 12:34:30 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='Models/Claims_model', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov14_12-34-30_babc6139389e', logging_f

In [9]:
# this allow us to generate sample patent claims 

from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import numpy as np

OUTPUT_DIR = "./Models/Claims_model"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
# we use the GPT-2 model to load our fine-tuned model and get the tokenization
tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
                                        
def generate(input_str, length=250, n=5):
  '''
  This is the main generation code using our model
  '''
  cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
  model.eval()
  with torch.no_grad():
    for i in range(length):
      outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
      loss, logits = outputs[:2]
      softmax_logits = torch.softmax(logits[0,-1], dim=0)
      next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
      cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
    return output_text

def choose_from_top(probs, n=5):
    '''
    This chooses the top n choices and probabilistically select one choice
    '''
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)
# print the generated text
generated_text = generate("This aparatus \n", n=5)
print(generated_text)

This aparatus 
(1) a method of forming a plurality of light emitting devices comprising: forming a plurality of light emitting devices on a substrate;  forming a first light emitting layer on a substrate, wherein the first light emitting layer is arranged in the first direction, and wherein the first light emitting layer is arranged in the second direction;  forming a plurality of light emitting devices on a substrate, wherein the plurality of light emitting devices are formed in the first direction and arranged in the second direction;  and forming a plurality of light emitting devices on an insulating film, wherein the plurality of light emitting devices are formed on the substrate and in the second direction. <EOS>
<BOS> 4.  The display device according to claim 3, wherein the first conductive layer comprises an organic material and the second conductive layer comprises a metal layer. <EOS>
<BOS> nan <EOS>
<BOS> 1.  A light emitting device comprising: a first substrate;  a second su

In [10]:
% ls output

ls: cannot access 'output': No such file or directory
